In [1]:
import requests
import pandas as pd
import pandasql as ps

In [2]:
request = requests.get('https://dapi.binance.com/dapi/v1/exchangeInfo').json()
info = {}
for symb in request['symbols']:
    if symb['pair'] in info:
        info[symb['pair']] += ', ' + symb['symbol']
    else:
        info[symb['pair']] = symb['symbol']
df_binance = pd.DataFrame(list(info.items()),columns = ['pair','symbol'])
df_binance

,pair,symbol
0,BTCUSD,"BTCUSD_PERP, BTCUSD_210625, BTCUSD_210924"
1,ETHUSD,"ETHUSD_PERP, ETHUSD_210625, ETHUSD_210924"
2,LINKUSD,"LINKUSD_PERP, LINKUSD_210625, LINKUSD_210924"
3,BNBUSD,"BNBUSD_PERP, BNBUSD_210625, BNBUSD_210924"
4,TRXUSD,TRXUSD_PERP
5,DOTUSD,"DOTUSD_PERP, DOTUSD_210625, DOTUSD_210924"
6,ADAUSD,"ADAUSD_PERP, ADAUSD_210625, ADAUSD_210924"
7,EOSUSD,EOSUSD_PERP
8,LTCUSD,"LTCUSD_PERP, LTCUSD_210625, LTCUSD_210924"
9,BCHUSD,"BCHUSD_PERP, BCHUSD_210625, BCHUSD_210924"


In [3]:
def get_okex_instruments(inst_types):
    info = {}
    for inst_type in inst_types:
        response = requests.request('GET', 'https://www.okex.com/api/v5/public/instruments', params={'instType':inst_type}).json()
        for symb in response['data']:
            if symb['uly'] in info:
                info[symb['uly']] += ', ' + symb['instId']
            else:
                info[symb['uly']] = symb['instId']
    df_okes = pd.DataFrame(list(info.items()),columns = ['uly','inst_id'])

    return df_okes

In [4]:
df_okex = get_okex_instruments(['FUTURES', 'SWAP'])
df_okex

,uly,inst_id
0,BTC-USD,"BTC-USD-210416, BTC-USD-210423, BTC-USD-210625..."
1,ETH-USD,"ETH-USD-210416, ETH-USD-210423, ETH-USD-210625..."
2,LTC-USD,"LTC-USD-210416, LTC-USD-210423, LTC-USD-210625..."
3,DOT-USD,"DOT-USD-210416, DOT-USD-210423, DOT-USD-210625..."
4,FIL-USD,"FIL-USD-210416, FIL-USD-210423, FIL-USD-210625..."
...,...,...
122,YFII-USDT,YFII-USDT-SWAP
123,ZEC-USDT,ZEC-USDT-SWAP
124,ZEN-USDT,ZEN-USDT-SWAP
125,ZIL-USDT,ZIL-USDT-SWAP


In [5]:
df_okex.uly = df_okex.uly.apply(lambda x: x.replace('-',''))
df_okex

,uly,inst_id
0,BTCUSD,"BTC-USD-210416, BTC-USD-210423, BTC-USD-210625..."
1,ETHUSD,"ETH-USD-210416, ETH-USD-210423, ETH-USD-210625..."
2,LTCUSD,"LTC-USD-210416, LTC-USD-210423, LTC-USD-210625..."
3,DOTUSD,"DOT-USD-210416, DOT-USD-210423, DOT-USD-210625..."
4,FILUSD,"FIL-USD-210416, FIL-USD-210423, FIL-USD-210625..."
...,...,...
122,YFIIUSDT,YFII-USDT-SWAP
123,ZECUSDT,ZEC-USDT-SWAP
124,ZENUSDT,ZEN-USDT-SWAP
125,ZILUSDT,ZIL-USDT-SWAP


In [6]:
sql_request = '''SELECT 
    df_binance.pair AS unify_name, df_binance.symbol AS binance_name,
    df_okex.inst_id AS okex_name
    FROM df_binance
    JOIN df_okex ON df_binance.pair=df_okex.uly
'''

adjacent_table = ps.sqldf(sql_request, locals())
adjacent_table

,unify_name,binance_name,okex_name
0,BTCUSD,"BTCUSD_PERP, BTCUSD_210625, BTCUSD_210924","BTC-USD-210416, BTC-USD-210423, BTC-USD-210625..."
1,ETHUSD,"ETHUSD_PERP, ETHUSD_210625, ETHUSD_210924","ETH-USD-210416, ETH-USD-210423, ETH-USD-210625..."
2,LINKUSD,"LINKUSD_PERP, LINKUSD_210625, LINKUSD_210924","LINK-USD-210416, LINK-USD-210423, LINK-USD-210..."
3,TRXUSD,TRXUSD_PERP,"TRX-USD-210416, TRX-USD-210423, TRX-USD-210625..."
4,DOTUSD,"DOTUSD_PERP, DOTUSD_210625, DOTUSD_210924","DOT-USD-210416, DOT-USD-210423, DOT-USD-210625..."
5,ADAUSD,"ADAUSD_PERP, ADAUSD_210625, ADAUSD_210924","ADA-USD-210416, ADA-USD-210423, ADA-USD-210625..."
6,EOSUSD,EOSUSD_PERP,"EOS-USD-210416, EOS-USD-210423, EOS-USD-210625..."
7,LTCUSD,"LTCUSD_PERP, LTCUSD_210625, LTCUSD_210924","LTC-USD-210416, LTC-USD-210423, LTC-USD-210625..."
8,BCHUSD,"BCHUSD_PERP, BCHUSD_210625, BCHUSD_210924","BCH-USD-210416, BCH-USD-210423, BCH-USD-210625..."
9,XRPUSD,"XRPUSD_PERP, XRPUSD_210625, XRPUSD_210924","XRP-USD-210416, XRP-USD-210423, XRP-USD-210625..."


In [7]:
adjacent_table = adjacent_table.sort_values(by=['unify_name']).reset_index(drop=True)
adjacent_table

,unify_name,binance_name,okex_name
0,ADAUSD,"ADAUSD_PERP, ADAUSD_210625, ADAUSD_210924","ADA-USD-210416, ADA-USD-210423, ADA-USD-210625..."
1,BCHUSD,"BCHUSD_PERP, BCHUSD_210625, BCHUSD_210924","BCH-USD-210416, BCH-USD-210423, BCH-USD-210625..."
2,BTCUSD,"BTCUSD_PERP, BTCUSD_210625, BTCUSD_210924","BTC-USD-210416, BTC-USD-210423, BTC-USD-210625..."
3,DOGEUSD,DOGEUSD_PERP,DOGE-USD-SWAP
4,DOTUSD,"DOTUSD_PERP, DOTUSD_210625, DOTUSD_210924","DOT-USD-210416, DOT-USD-210423, DOT-USD-210625..."
5,EOSUSD,EOSUSD_PERP,"EOS-USD-210416, EOS-USD-210423, EOS-USD-210625..."
6,ETCUSD,ETCUSD_PERP,"ETC-USD-210416, ETC-USD-210423, ETC-USD-210625..."
7,ETHUSD,"ETHUSD_PERP, ETHUSD_210625, ETHUSD_210924","ETH-USD-210416, ETH-USD-210423, ETH-USD-210625..."
8,FILUSD,FILUSD_PERP,"FIL-USD-210416, FIL-USD-210423, FIL-USD-210625..."
9,LINKUSD,"LINKUSD_PERP, LINKUSD_210625, LINKUSD_210924","LINK-USD-210416, LINK-USD-210423, LINK-USD-210..."


In [8]:
csv = adjacent_table.to_csv(index=False, header=False)
print(csv)

ADAUSD,"ADAUSD_PERP, ADAUSD_210625, ADAUSD_210924","ADA-USD-210416, ADA-USD-210423, ADA-USD-210625, ADA-USD-210924, ADA-USD-SWAP"
BCHUSD,"BCHUSD_PERP, BCHUSD_210625, BCHUSD_210924","BCH-USD-210416, BCH-USD-210423, BCH-USD-210625, BCH-USD-210924, BCH-USD-SWAP"
BTCUSD,"BTCUSD_PERP, BTCUSD_210625, BTCUSD_210924","BTC-USD-210416, BTC-USD-210423, BTC-USD-210625, BTC-USD-210924, BTC-USD-SWAP"
DOGEUSD,DOGEUSD_PERP,DOGE-USD-SWAP
DOTUSD,"DOTUSD_PERP, DOTUSD_210625, DOTUSD_210924","DOT-USD-210416, DOT-USD-210423, DOT-USD-210625, DOT-USD-210924, DOT-USD-SWAP"
EOSUSD,EOSUSD_PERP,"EOS-USD-210416, EOS-USD-210423, EOS-USD-210625, EOS-USD-210924, EOS-USD-SWAP"
ETCUSD,ETCUSD_PERP,"ETC-USD-210416, ETC-USD-210423, ETC-USD-210625, ETC-USD-210924, ETC-USD-SWAP"
ETHUSD,"ETHUSD_PERP, ETHUSD_210625, ETHUSD_210924","ETH-USD-210416, ETH-USD-210423, ETH-USD-210625, ETH-USD-210924, ETH-USD-SWAP"
FILUSD,FILUSD_PERP,"FIL-USD-210416, FIL-USD-210423, FIL-USD-210625, FIL-USD-210924, FIL-USD-SWAP"
LINKUSD,"LINKUSD_PERP